In [1]:
import json
import zipfile
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
from pathlib import Path
data_dir = Path('.')
data_dir.mkdir(exist_ok = True)
file_path = data_dir / Path('CVR1.zip')
dest_path = file_path

In [3]:
zip_1 = zipfile.ZipFile(dest_path, 'r')
